# Preparing Data for Mezonet ET validation

This notebook contains code for processing field measurement in Minnesota, USA used in the current work for additional comparison (Appendix A).

In [1]:
import re
import numpy as np
import pandas as pd

import shapely
import geopandas as gpd

from os import listdir
from os.path import join

import matplotlib.pyplot as plt

In [2]:
files_19_21 = sorted(listdir('../data_v02/validation/2019_2021'))
files_22 = sorted(listdir('../data_v02/validation/2022'))

In [3]:
files_22

['Becker_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Clarissa_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Hastings_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Hubbard_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'LittleFalls_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Ottertail_CR1000_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'ParkersPrairie_CR1000_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Perham_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'PinePoint_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Rice_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Staples_CR1000X_AllData_Daily_1-1-22_thru_12-31-22.xlsx',
 'Wadena_CR1000_AllData_Daily_1-1-22_thru 12-31-22.xlsx',
 'Westport_CR1000_AllData_Daily_1-1-22_thru 12-31-22.xlsx']

In [4]:
coords = pd.read_csv('../data_v02/validation/Location_Coordinates.csv')
coords.loc[6, 'Location name'] = 'Parkers Prairie'
coords.loc[8, 'Location name'] = 'Pine Point'
coords

,Location name,Latitude,Longitude
0,Becker,45.34399,-93.85014
1,Clarissa,46.11155,-94.90583
2,Dakota,44.68768,-92.85985
3,Hubbard,46.82059,-94.99579
4,Little Falls,45.93214,-94.25144
5,Otter Tail,46.41095,-95.57431
6,Parkers Prairie,46.16941,-95.35602
7,Perham,46.61049,-95.60188
8,Pine Point,47.01272,-95.37151
9,Rice,45.79384,-94.26182


In [5]:
sites, dfs = {}, []
cols = ['TIMESTAMP', 'ETos_daily_in', 'ETrs_daily_in']
for file1, file2 in zip(files_19_21, files_22):
    site = file1.split('_')[0]
    if site == 'Hastings': # ASSUMING THAT Hustings == Dakota; all the other site resembles site names from coords dataframe
        site = 'Dakota'
    elif site=='Ottertail':
        site = 'OtterTail'
    site = re.sub(r'(?<!^)(?=[A-Z])', ' ', site)
    try:
        df1 = pd.read_excel(join('../data_v02/validation/2019_2021', file1), skiprows=(1,2))
        df1.TIMESTAMP = pd.to_datetime(df1.TIMESTAMP).dt.date
    except:
        try:
            df1 = pd.read_excel(join('../data_v02/validation/2019_2021', file1), skiprows=(0,2,3))
            df1.TIMESTAMP = pd.to_datetime(df1.TIMESTAMP).dt.date
        except Exception:
            df1 = []
    
    
    try:
        df2 = pd.read_excel(join('../data_v02/validation/2022', file2), skiprows=(1,2))
        df2.TIMESTAMP = pd.to_datetime(df2.TIMESTAMP).dt.date
    except Exception:
        try:
            df2 = pd.read_excel(join('../data_v02/validation/2022', file2), skiprows=(0,2,3))
            df2.TIMESTAMP = pd.to_datetime(df2.TIMESTAMP).dt.date
        except:
            df2 = []
    
    
    df = pd.concat([df1[cols], df2[cols]], axis=0).reset_index(drop=True)
    df['ETos_daily_in'] = pd.to_numeric(df['ETos_daily_in'], errors='coerce')
    df['ETrs_daily_in'] = pd.to_numeric(df['ETrs_daily_in'], errors='coerce')
#     df['ETos_daily_in'] *= 721.25 #--> LE (W m-2)
#     df['ETrs_daily_in'] *= 721.25 #--> LE (W m-2)
    sites[site] = df

    df['site'] = [site] * len(df)
    df['lat'] = [coords[coords['Location name']==site]['Latitude'].item()] * len(df)
    df['lon'] = [coords[coords['Location name']==site]['Longitude'].item()] * len(df)
    dfs.append(df)
df = pd.concat(dfs, axis=0).reset_index(drop=True)

In [7]:
df.to_csv('../data_v02/validation/validation_dataset.csv', index=None)

In [8]:
df

,TIMESTAMP,ETos_daily_in,ETrs_daily_in,site,lat,lon
0,2019-01-01,0.007,0.009,Becker,45.34399,-93.85014
1,2019-01-02,0.013,0.019,Becker,45.34399,-93.85014
2,2019-01-03,0.023,0.033,Becker,45.34399,-93.85014
3,2019-01-04,0.024,0.031,Becker,45.34399,-93.85014
4,2019-01-05,0.025,0.036,Becker,45.34399,-93.85014
...,...,...,...,...,...,...
18278,2022-12-27,0.013,0.018,Westport,45.71509,-95.17179
18279,2022-12-28,0.009,0.010,Westport,45.71509,-95.17179
18280,2022-12-29,0.004,0.004,Westport,45.71509,-95.17179
18281,2022-12-30,0.011,0.013,Westport,45.71509,-95.17179
